# Day 8, Part 1 - intro to ipyvolume
We'll start our journey into the 3RD DIMENSION with the package ```ipyvolume```

In [1]:
# if you don't get it:
#!pip install ipyvolume
# note: you may need:
#!jupyter nbextension enable --py --sys-prefix ipyvolume
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

# or you can do:
#!conda install -c conda-forge ipyvolume


import ipyvolume

Let's do a quick look at something with some random 3D data:

In [2]:
import numpy as np
x, y, z = np.random.random((3, 10000))
ipyvolume.quickscatter(x, y, z, size=1, marker="sphere")

Easy peasy!  Let's read in our simulation data and plot this!

In [3]:
from sys import path
path.append('../lesson02/')

from hermite_library import do_hermite

In [4]:
star_mass = 1.0 # stellar mass in Msun
planet_masses = np.array( [1.0, 0.05] ) # planet masses in Mjupiter

# [x,y,z] coords for each planet in AU
# NOTE: no z-coords!  These will be set to zero later on if you make them non-zero
planet_initial_position = np.array([ [5.0, 0.0, 6.0], 
                                     [10.0, 0.0, 3.0] ])

# planet's velocity at each position in km/s
# NOTE: no z-velocities!  These will be set to zero later on if you make them non-zero
planet_initial_velocity = np.array([ [0.0, 10.0, 1.0], 
                                     [0.0, -5.0, 0.0]])

In [5]:
# h is for hermite!
r_h, v_h, t_h, e_h = do_hermite(star_mass, 
                                planet_masses, 
                                planet_initial_position, 
                                planet_initial_velocity, 
                               tfinal=200, Nsteps=8800, 
                               threeDee=True)

In [6]:
# we'll have to reformat a bit for plotting
# right now, just all as one color
x = r_h[:,0,:].ravel()
y = r_h[:,1,:].ravel()
z = r_h[:,2,:].ravel()
ipyvolume.quickscatter(x, y, z, 
                       size=1, marker="sphere")

Let's make things a little more complicated and allow us to take a look at each orbit:

In [7]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(r_h.shape[0]):
    ipyvolume.scatter(r_h[i,0,:],
                      r_h[i,1,:],
                      r_h[i,2,:], 
                      color=colors[i], 
                     marker='sphere')
ipyvolume.show()

So, this is pretty cool - we can now see how the orbits "precess" during their evolution and we can check out these shapes in 3D.

Note we can also plot more abstract spaces in 3D - like velocity space:

In [8]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.scatter(v_h[i,0,:],
                      v_h[i,1,:],
                      v_h[i,2,:], 
                      color=colors[i],
                     marker='sphere')
ipyvolume.show()

With this we can see how "jumpy" the velocity changes can get - this may be a numerical effect that is causing the precession of the orbits, or just how things are!

Ok, we can also show velocity by little vectors:

In [9]:
ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.quiver(r_h[i,0,:],
                      r_h[i,1,:],
                      r_h[i,2,:],
                     v_h[i,0,:],
                      v_h[i,1,:],
                      v_h[i,2,:], 
                      color=colors[i])
ipyvolume.show()

So clearly the above is pointless - while it looks cool the arrows are too big and there are too many of them!  We can change this by taking "X" number of points:

In [10]:
step = 600
# also, length of arrays
N = v_h.shape[2]

ipyvolume.figure()
colors = ['red', 'blue', 'green']
for i in range(v_h.shape[0]):
    ipyvolume.quiver(r_h[i,0,0:N:step],
                      r_h[i,1,0:N:step],
                      r_h[i,2,0:N:step],
                     v_h[i,0,0:N:step],
                      v_h[i,1,0:N:step],
                      v_h[i,2,0:N:step], 
                      color=colors[i])
ipyvolume.show()

Now we can see a bit more about the motion - that their directions are opposite of eachother for example.  And that the central mass only moves slightly and around its center as well.

## Animation
Let's now figure out how to make an animation, and then save it for ourselves!  To do this, we'll need to format our data specifically as (time, position):

In [11]:
# for example, for particle 0:
r_h[:,0,:].T.shape

(8800, 3)

In [12]:
step = 10
# also, length of arrays
N = v_h.shape[2]

r = r_h[:,:,0:N:step]
v = v_h[:,:,0:N:step]

r_h.shape, r.shape, r[:,2,:].T.shape

((3, 3, 8800), (3, 3, 880), (880, 3))

In [13]:
# have to format color as well
#colors = np.empty((0,3))
color = [(1,0,0), (0,0,1), (0,1,0)]

#colors = np.array([])
colors = []
for i in range(r.shape[2]):
    colors.append(color)
    
colors = np.array(colors)

# order should be (times, points, colors)
colors = np.transpose(colors, (0, 2, 1)) # flip the last axes

colors.shape

(880, 3, 3)

In [14]:
ipyvolume.figure()

s = ipyvolume.scatter(r[:,0,:].T, r[:,1,:].T, r[:,2,:].T, 
                      marker='sphere', 
                     color=colors)

ani = ipyvolume.animation_control(s, interval=200)

ipyvolume.show()

Note that we can only use the ```animation_control``` function on scatter plots or quiver plots, so we can't add lines or anything here.  Perhaps in a future release of ```ipyvolume```!

## Saving animations as gifs - DON'T DO THIS ONE
Now if you have a nice animation you like, you can save it as a gif or other movie file!  To do that we have to set a "view" for our camera and then play our movie.  We can start with something simple. Suppose we like our current view above, let's try saving this with:

In [15]:
myView = ipyvolume.view()

myView

(0.0, 0.0, 2.0)

Then we generate a function that will set this view for all frames.  It will take in a few parameters that we will ignore:

In [16]:
def set_view(figure, framenr, fraction):
    ipyvolume.view(myView[0], myView[1], myView[2])

In [17]:
#def set_view(figure, framenr, fraction):
#    ipyvolume.view(fraction*360, (fraction - 0.5) * 180, distance=2 + fraction*2)
#    s.size = size * (2+0.5*np.sin(fraction * 6 * np.pi))


In [23]:
#ipyvolume.movie('myMovie.gif', set_view, fps=20, frames=40)
#convert -delay 5.0 -loop 0 /var/folders/4h/rklzvjw

In [18]:
#!open myMovie.gif

ERROR! Session/line number was not unique in database. History logging moved to new session 287


Note: the above does nothing.   This is because the animations only are for changing angles of things, so bugger it.

## Embedding

In [25]:
import ipywidgets

In [26]:
myBox = ipywidgets.VBox([ani])
myBox

TraitError: Element of the 'children' trait of a VBox instance must be a Widget, but a value of None <class 'NoneType'> was specified.

In [19]:
#ipyvolume.embed.embed_html("testEmbed.html", ani, offline=True, devmode=True)


In [20]:
#!open testEmbed.html

The file /Users/jillnaiman1/csci-p-14110/lesson08/testEmbed.html does not exist.


In [24]:
#ipyvolume.embed.embed_html?